In [1]:
import dask
import ee
import geemap
import geopandas as gpd
import numpy as np
import pandas as pd
import plotly.express as px
from dask import compute, delayed
from dask.distributed import Client
from tqdm.notebook import tqdm

In [2]:
ee.Authenticate()

True

In [3]:
ee.Initialize()

In [4]:
start_date = "2000-01-01"
end_date = "2000-02-28"

In [5]:
spain_map = ee.FeatureCollection("FAO/GAUL/2015/level0").filter(
    ee.Filter.eq("ADM0_NAME", "Spain")
)

In [6]:
spain_provinces = gpd.read_file("../data/maps/spain_map.geojson")

In [7]:
spain_provinces = spain_provinces[
    spain_provinces["community"]
    != "Gibraltar. Territorio no asociado a ninguna autonomía"
]

In [8]:
terra_climate_dataset = (
    ee.ImageCollection("IDAHO_EPSCOR/TERRACLIMATE")
    .filterBounds(spain_map)
    .filterDate(start_date, end_date)
)

In [9]:
def get_monthly_data(feature):
    province_name = feature["properties"]["province"]
    province_geometry = ee.Geometry(feature["geometry"])

    def spatial_aggregation(image):
        bands = image.select(["pr", "pet"])

        # Apply reduceRegion once to get both mean values
        mean_values_dict = bands.reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=province_geometry,
            scale=4638.3,
        )

        date = image.date().format("YYYY-MM-dd")

        feature = ee.Feature(
            None,
            {
                "province": province_name,
                "date": date,
                "avg_precipitation": mean_values_dict.get("pr"),
                "avg_pet": mean_values_dict.get("pet"),
            },
        )

        return feature

    return terra_climate_dataset.map(spatial_aggregation)

In [10]:
def get_data_as_dataframe(feature):
    ee.Initialize()
    monthly_data = get_monthly_data(feature).getInfo()
    df = pd.DataFrame([r["properties"] for r in monthly_data["features"]])
    return df

# Dask stuff:

In [11]:
features_list = list(spain_provinces.iterfeatures())

In [12]:
client = Client()

C:\Users\Reslan Al Tinawi\Desktop\CS913-dissertation-project\cs913-dissertation-project\lib\site-packages\distributed\node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 52730 instead
  warnings.warn(


In [13]:
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:52730/status,
Dashboard: http://127.0.0.1:52730/status,Workers: 4
Total threads: 16,Total memory: 15.71 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:52731,Workers: 4
Dashboard: http://127.0.0.1:52730/status,Total threads: 16
Started: Just now,Total memory: 15.71 GiB
Comm: tcp://127.0.0.1:52757,Total threads: 4
Dashboard: http://127.0.0.1:52758/status,Memory: 3.93 GiB
Nanny: tcp://127.0.0.1:52734,


In [14]:
delayed_results = [delayed(get_data_as_dataframe)(feature) for feature in features_list]

In [15]:
results = compute(*delayed_results)

C:\Users\Reslan Al Tinawi\Desktop\CS913-dissertation-project\cs913-dissertation-project\lib\site-packages\distributed\client.py:3245: UserWarning: Sending large graph of size 31.44 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(


In [ ]:
# client.close()

In [17]:
pd.concat(results)

,avg_pet,avg_precipitation,date,province
0,142.446771,22.252368,2000-01-01,Araba/Álava
1,350.056457,23.672784,2000-02-01,Araba/Álava
0,280.206665,21.396714,2000-01-01,Albacete
1,511.694386,0.332510,2000-02-01,Albacete
0,351.700535,23.740549,2000-01-01,Alacant/Alicante
...,...,...,...,...
1,598.714286,0.000000,2000-02-01,Territorios no asociados a ninguna provincia
0,717.599079,48.837677,2000-01-01,Las Palmas
1,919.577949,18.840789,2000-02-01,Las Palmas
0,702.556022,72.590353,2000-01-01,Santa Cruz de Tenerife
